In [1]:
import torch.utils.data
import random
from os import path
import numpy as np
import torch
import torchvision.transforms as transforms
import dataloaders
from matplotlib import pyplot as plt
import torchvision.utils as vutils

from networks import dcgan
from networks import wdcgan_gp
from networks import sndcgan
from networks import rdcgan
from networks import cdcgan
from networks import wcdcgan_gp
from networks import sncdcgan
from networks import rcdcgan

from networks.utils import compute_kid_score
from networks.utils import plot_grid
from sample_and_visualize import sample_and_visualize

In [2]:
USE_CUDA = True
if USE_CUDA and torch.cuda.is_available():
    device = 'cuda'  #alt: "cuda:0"
else:
    device = 'cpu'

PIN_MEMORY = True
USE_MULTI_PROCESS_DATA_LOADING = True
if USE_MULTI_PROCESS_DATA_LOADING:
    NUM_THREADS = torch.get_num_threads()
    print("Number of Threads: ", NUM_THREADS)
else:
    NUM_THREADS = 0
    print("Not using Multi-Process Data Loading.")

SEED = 0
random.seed(SEED)
torch.manual_seed(SEED);

### Data
DATA_PATH = path.join("data", "tmdb-64")
TABLE_PATH = path.join("data", "tmdb-movies-220915-clean.pkl")

Number of Threads:  8


In [3]:
### Architecture
IMAGE_SIZE = 64
IMAGE_SIZE_RATIO = 3
NUM_IMG_CHANNELS = 3
NUM_NOISE_VEC_CHANNELS = 100
NUM_FEATURE_VEC_CHANNELS = 37
BASE_NUM_OUT_CHANNELS_G_non_conditional = 64
BASE_NUM_OUT_CHANNELS_G_conditional = 128
BASE_NUM_OUT_CHANNELS_D = 64
D_NORM_LAYER_TYPE = "instance"
PADDING_MODE = "reflect"
COLORMODE="RGB"

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Poster subset:
GENRE = None
GENRE_LOGIC = 'and'
OG_LANG = None
YEAR = None
RUNTIME = (40, np.inf)
MAX_NUM = None
SORT = None

BATCH_SIZE = 100
mode = "generated"
iterations = 5
seed = 79469812346923649

genres = ["is_thriller", "is_western", "is_music"]
languages = ["lang_en", "lang_ar", "lang_ja", "lang_zh", "lang_ru"]

In [4]:
poster_dataset = dataloaders.PosterDataset(table_path=TABLE_PATH, img_root_path=DATA_PATH,
                                           img_transform=transforms.Compose([
                                               transforms.ToTensor(),
                                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]),
                                           colormode=COLORMODE,
                                           img_in_ram=False,
                                           genre=GENRE, genre_logic=GENRE_LOGIC, og_lang=OG_LANG, year=YEAR,
                                           runtime=RUNTIME,
                                           max_num=MAX_NUM, sort=SORT)

posterloader = torch.utils.data.DataLoader(poster_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_THREADS,
                                           pin_memory=PIN_MEMORY)

In [5]:
import pickle
import glob
def evaluate_kid(model_name, model_dir, generator, conditional=False):
    print(model_name)
    models = [model for model in glob.glob(f"{model_dir}/*.pt")]
    num_features = None
    if conditional:
        num_features = NUM_FEATURE_VEC_CHANNELS
    print("KID score:\n")
    means = []
    stds = []
    for model in models:
        print(model)
        generator.load_state_dict(torch.load(model)['generator_model_state_dict'])
        generator.eval()
        mean = compute_kid_score(generator, posterloader, mode, iterations, BATCH_SIZE, NUM_NOISE_VEC_CHANNELS,
                          IMAGE_SIZE_RATIO, device, num_features)
        means.append(mean)
        #stds.append(std)
    kid_scores = {"means": means, "stds": stds}
    with open(path.join(model_dir, f"{model_name}_kid.pkl"), 'wb') as f:
        pickle.dump(kid_scores, f)
    fig = plt.figure(figsize=(10, 5))
    plt.plot(range(len(means)), means)
    plt.show()
    fig.savefig(path.join(model_dir, f"{model_name}_kid.png"))

In [6]:
generator, _ = dcgan.create_gan(img_size=IMAGE_SIZE, img_ratio=IMAGE_SIZE_RATIO,
                                            num_img_channels=NUM_IMG_CHANNELS,
                                            num_noise_vec_channels=NUM_NOISE_VEC_CHANNELS,
                                            base_num_out_channels_g=BASE_NUM_OUT_CHANNELS_G_non_conditional,
                                            base_num_out_channels_d=BASE_NUM_OUT_CHANNELS_D, padding_mode=PADDING_MODE,
                                            device=device)
evaluate_kid("DCGAN", "output-runs\\dcgan-output\\2022-09-26_21-16-09", generator, conditional=False)

DCGAN
KID score:

output-runs\dcgan-output\2022-09-26_21-16-09\gan_at_epoch_0000.pt


  0%|          | 0/5 [00:00<?, ?it/s]D:\ProgramData\Anaconda3\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `FrechetInceptionDistance` will save all extracted features in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
D:\ProgramData\Anaconda3\lib\site-packages\scipy\linalg\_matfuncs_sqrtm.py:189: RuntimeWarning: invalid value encountered in double_scalars
  arg2 = norm(X.dot(X) - A, 'fro')**2 / norm(A, 'fro')
 20%|██        | 1/5 [00:14<00:59, 14.92s/it]

12.205965042114258


 40%|████      | 2/5 [00:27<00:41, 13.70s/it]

16.789871215820312


...

 80%|████████  | 4/5 [00:53<00:12, 12.96s/it]

15.258699417114258


100%|██████████| 5/5 [01:05<00:00, 13.18s/it]

17.34008026123047


AttributeError: 'float' object has no attribute 'item'

In [ ]:
generator, _ = wdcgan_gp.create_gan(num_img_channels=NUM_IMG_CHANNELS,
                                             num_noise_vec_channels=NUM_NOISE_VEC_CHANNELS,
                                             base_num_out_channels_g=BASE_NUM_OUT_CHANNELS_G_non_conditional,
                                             base_num_out_channels_d=BASE_NUM_OUT_CHANNELS_D,
                                             d_norm_layer_type=D_NORM_LAYER_TYPE, padding_mode=PADDING_MODE,
                                             device=device)
evaluate_kid("WDCGAN", "output-runs\\wdcgan-output\\2022-09-25_04-48-49", generator, conditional=False)

In [ ]:
generator, _ = sndcgan.create_gan(num_img_channels=NUM_IMG_CHANNELS,
                                              num_noise_vec_channels=NUM_NOISE_VEC_CHANNELS,
                                              base_num_out_channels_g=BASE_NUM_OUT_CHANNELS_G_non_conditional,
                                              base_num_out_channels_d=BASE_NUM_OUT_CHANNELS_D, padding_mode=PADDING_MODE,
                                              device=device)
evaluate_kid("SNDCGAN", "output-runs\\sndcgan-output\\2022-09-27_04-18-01", generator, conditional=False)

In [ ]:
generator, discriminator = rdcgan.create_gan(img_size=IMAGE_SIZE, img_ratio=IMAGE_SIZE_RATIO,
                                            num_img_channels=NUM_IMG_CHANNELS,
                                            num_noise_vec_channels=NUM_NOISE_VEC_CHANNELS,
                                            base_num_out_channels_g=BASE_NUM_OUT_CHANNELS_G_non_conditional,
                                            base_num_out_channels_d=BASE_NUM_OUT_CHANNELS_D, padding_mode=PADDING_MODE,
                                            device=device)
#evaluate_kid("RaDCGAN", "output-runs\\radcgan-output\\2022-09-25_04-48-49", generator, conditional=False)

In [ ]:
generator, _ = cdcgan.create_gan(num_img_channels=NUM_IMG_CHANNELS,
                                             num_noise_vec_channels=NUM_NOISE_VEC_CHANNELS,
                                             base_num_out_channels_g=BASE_NUM_OUT_CHANNELS_G_conditional,
                                             base_num_out_channels_d=BASE_NUM_OUT_CHANNELS_D, padding_mode=PADDING_MODE,
                                             num_feature_vec_channels=NUM_FEATURE_VEC_CHANNELS, device=device)
evaluate_kid("cDCGAN", "output-runs\\cdcgan-output\\2022-09-27_00-55-04", generator, conditional=True)

In [ ]:
generator, _ = wcdcgan_gp.create_gan(num_img_channels=NUM_IMG_CHANNELS, num_noise_vec_channels=NUM_NOISE_VEC_CHANNELS,
                                    base_num_out_channels_g=BASE_NUM_OUT_CHANNELS_G_conditional,
                                    base_num_out_channels_d=BASE_NUM_OUT_CHANNELS_D, padding_mode=PADDING_MODE,
                                    num_feature_vec_channels=NUM_FEATURE_VEC_CHANNELS,
                                    d_norm_layer_type=D_NORM_LAYER_TYPE, device=device)
evaluate_kid("WcDCGAN", "output-runs\\wcdcgan-output\\2022-09-25_23-40-46", generator, conditional=True)

In [ ]:
generator, _ = sncdcgan.create_gan(num_img_channels=NUM_IMG_CHANNELS,
                                               num_noise_vec_channels=NUM_NOISE_VEC_CHANNELS,
                                               base_num_out_channels_g=BASE_NUM_OUT_CHANNELS_G_conditional,
                                               base_num_out_channels_d=BASE_NUM_OUT_CHANNELS_D,
                                               padding_mode=PADDING_MODE,
                                               num_feature_vec_channels=NUM_FEATURE_VEC_CHANNELS, device=device)
evaluate_kid("SNcDCGAN", "output-runs\\sncdcgan-output\\2022-09-27_06-53-59", generator, conditional=True)

In [ ]:
generator, _ = rcdcgan.create_gan(img_size=IMAGE_SIZE, img_ratio=IMAGE_SIZE_RATIO,
                                              num_img_channels=NUM_IMG_CHANNELS,
                                              num_noise_vec_channels=NUM_NOISE_VEC_CHANNELS,
                                            num_feature_vec_channels=NUM_FEATURE_VEC_CHANNELS,
                                              base_num_out_channels_g=BASE_NUM_OUT_CHANNELS_G_conditional,
                                              base_num_out_channels_d=BASE_NUM_OUT_CHANNELS_D,
                                              padding_mode=PADDING_MODE,
                                              device=device)
#evaluate_kid("RacDCGAN", "output-runs\\racdcgan-output\\2022-09-25_04-48-49", generator, conditional=True)